In [4]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [7]:
import re
import random
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

In [8]:
from utils import UtilsKy 

In [27]:
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype='str', encoding='cp1251')
db_test =  pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype='str', encoding='cp1251')

In [28]:
COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'longitude', 'latitude', 'phone_2_norm'] 

In [31]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [32]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [46]:
mask = train.longitude.isnull() | train.latitude.isnull()
df = train[mask]
df.head()

,bin,amount,bank_currency,hour,day_of_week,longitude,latitude,phone_2_norm


In [37]:
# mean _long 
# mean_lat 
mean_long = train.longitude.mean(skipna=True)
mean_lat = train.latitude.mean(skipna=True)

In [39]:
mean_long

-92.5332586154281

In [42]:
mean_lat

36.902375778905636

In [47]:
train.longitude.fillna(mean_long, inplace=True)
test.longitude.fillna(mean_long, inplace=True)

train.latitude.fillna(mean_lat, inplace=True)
test.latitude.fillna(mean_lat, inplace=True)
#

In [86]:
from sklearn.decomposition import IncrementalPCA
# X = np.array([[-1, -1, 7], [-2, -1, 3], [-3, -2, 5], [1, 1,2], [2, 1,4], [3, 2,4]])
ipca = IncrementalPCA(n_components=2, batch_size=3)

In [87]:
# X = list(zip(db_teach.longitude.values, db_teach.latitude.values))
# X = list(zip(db_teach.longitude.values, db_teach.latitude.values))

In [88]:
X = [ [x_long, x_lat] for x_long, x_lat in zip(train.longitude.values, train.latitude.values) ] 
Y = [ [y_long, y_lat] for y_long, y_lat in zip(test.longitude.values, test.latitude.values) ] 

In [89]:
ipca = IncrementalPCA(n_components=1,batch_size=3)

In [91]:
ipca.fit(X)

IncrementalPCA(batch_size=3, n_components=1)

In [92]:
X_transform = ipca.transform(X)

In [93]:
X_pca_distance =[ x[0]for x in  X_transform]

In [94]:
Y_transform = ipca.transform(Y)

In [95]:
Y_pca_distance =[ y[0] for y in  Y_transform]

In [97]:
train['pca_distance'] = X_pca_distance
test['pca_distance'] = Y_pca_distance

In [98]:
train.head()

,bin,amount,bank_currency,hour,day_of_week,longitude,latitude,phone_2_norm,pca_distance
0,547087,1.00,112,9,3,9.1833,48.7667,55,94.709858
1,517148,25.00,840,12,3,-159.3553,22.0837,8,-60.317973
2,546540,21.18,840,15,3,-97.0000,38.0000,40,-4.597107
3,510805,200.00,840,15,3,-122.2420,37.8116,10,-28.846150
4,544731,2.38,76,15,3,-43.2333,-22.9000,19,63.644737
